Note to self 6/4-22:
Totalt antall stemmer godt med PSAK120.
Alt med DP stemmer bra med excel, men ikke så bra med PSAK120.
Automatisk stemmer så bra at autograd blir det samme på figur 1, men litt unøyaktighet her også.


For the future 22/4-22:
Når vi joiner med `PEN.T_SKJEMA` forsvinner (nesten) alle ikke-selvbetjente saker. Teste ut andre typer join for å konservere antall saker. Hvordan håndtere resulterende nullverdier fra `PEN.T_SKJEMA`?
Kan være nødvendig å se mer på `OPPRETTET_AV`; mye der som kommer fra batch.

In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()
resource_name = f"projects/knada-gcp/secrets/vebjorns-hemmelighet/versions/latest"
secret = secrets.access_secret_version(name=resource_name)
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

ORACLE_HOST = '10.53.136.15'
ORACLE_PORT = '1521'
ORACLE_SERVICE = 'pen'

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

In [ ]:
df_v3 = pandas_utils.pandas_from_sql('../sql/v3.sql', con)

In [ ]:
con.close()

In [ ]:
df = df_v3.copy()
# Add "ÅR-MÅNED" TO FRONT
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

df["STØNADSOMRÅDE"] = df.STØNADSOMRÅDE.apply(pesys_utils.map_stonad)

In [ ]:
df_ap = df.copy().loc[df.STØNADSOMRÅDE=="Alderspensjon"]
df_ap = df_ap.loc[df_ap.BATCH_FLAGG == "ikke opprettet av batch"]

In [ ]:
df_ap = df_ap.merge(df_ap.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
                left_on="ÅR-MÅNED",
                right_on="ÅR-MÅNED",
                suffixes=(""," TOTALT")
        )

In [ ]:
df_ap["ANDEL"] = df_ap.ANTALL.divide(df_ap["ANTALL TOTALT"])

In [ ]:
df_ap.head(10)

# Prep for plott

In [ ]:
nav_colors = (
    "#0067C5",
    "#BA3A26",
    "#06893A",
    "#634689",
    "#FF9100",
    "#66CBEC",
    "#F0C419",
)
pastel = px.colors.qualitative.Pastel

figs = {}

### Automatiseringsgrad

In [ ]:
df_auto = df_ap.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_auto = df_auto[df_auto.AUTOMATISERING == "AUTO"].reset_index(drop=True)
df_auto["ANDEL_PROSENT"] = df_auto["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_auto.tail()

In [ ]:
n_months = 17

#df_plot = df_auto.iloc[-n_months:-1,:]
df_plot = df_auto.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[0], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisk", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[0.2,0.8], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs["autograd"] = fig

### Selvbetjening

In [ ]:
df_selv = df_ap.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "OPPRETTET_AV", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_selv = df_selv[df_selv.OPPRETTET_AV == "bruker"].reset_index(drop=True)
df_selv["ANDEL_PROSENT"] = df_selv["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_selv.tail()

In [ ]:
#df_plot = df_selv.iloc[-n_months:-1,:]
df_plot = df_selv.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[1], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Selvbetjent", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel selvbetjent", secondary_y=True, range=[0.5,1], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs["selvbetjening"] = fig

# Teste andre idéer

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_v2 = pandas_utils.pandas_from_sql('../sql/v2.sql', con)

con.close()

In [ ]:
df_v2.tail(10)

# Automatisering av selvbetjente saker

In [ ]:
df = df_v2.copy()
df = df.loc[df.ÅR >= 2021]
# Add "ÅR-MÅNED" TO FRONT
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

In [ ]:
df_selv = df.loc[df["OPPRETTET_AV"] == "bruker"]
df_selv = df_selv.loc[df_selv["BATCH_FLAGG"] == "ikke opprettet av batch"]

In [ ]:
df_selv_auto = df_selv.groupby(["ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
fig = px.bar(df_selv_auto, "ÅR-MÅNED", "ANTALL", color="AUTOMATISERING", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
fig.show()

figs["selv_auto"] = fig

In [ ]:
df_ikkeselv = df.loc[df["OPPRETTET_AV"] == "saksbehandler"]
df_ikkeselv = df_ikkeselv.loc[df_ikkeselv["BATCH_FLAGG"] == "ikke opprettet av batch"]
df_ikkeselv_auto = df_ikkeselv.groupby(["ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
fig = px.bar(df_ikkeselv_auto, "ÅR-MÅNED", "ANTALL", color="AUTOMATISERING", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
fig.show()

figs["ikkeselv_auto"] = fig

# Utlandstilsnitt

In [ ]:
df_man = df.loc[df["AUTOMATISERING"] == "MAN"]

In [ ]:
df_man_utland = df_man.groupby(["ÅR-MÅNED", "UTLANDSTILSNITT"], as_index=False).ANTALL.sum()

fig = px.bar(df_man_utland, "ÅR-MÅNED", "ANTALL", color="UTLANDSTILSNITT", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["man_utland"] = fig

In [ ]:
df_auto = df.loc[df["AUTOMATISERING"] == "AUTO"]

In [ ]:
df_auto_utland = df_auto.groupby(["ÅR-MÅNED", "UTLANDSTILSNITT"], as_index=False).ANTALL.sum()

fig = px.bar(df_auto_utland, "ÅR-MÅNED", "ANTALL", color="UTLANDSTILSNITT", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["auto_utland"] = fig

# Opprettet av batch

In [ ]:
df_man_batch = df_man.groupby(["ÅR-MÅNED", "BATCH_FLAGG"], as_index=False).ANTALL.sum()

fig = px.bar(df_man_batch, "ÅR-MÅNED", "ANTALL", color="BATCH_FLAGG", color_discrete_sequence=nav_colors[2:])
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["man_batch"] = fig
fig.show()

In [ ]:
df_auto_batch = df_auto.groupby(["ÅR-MÅNED", "BATCH_FLAGG"], as_index=False).ANTALL.sum()

fig = px.bar(df_auto_batch, "ÅR-MÅNED", "ANTALL", color="BATCH_FLAGG", color_discrete_sequence=nav_colors[2:])
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["auto_batch"] = fig
fig.show()

In [ ]:
for key, value in stonads_mapper.items():
    print(key, value)

# Sjekk plott

In [ ]:
for _, fig in figs.items():
    fig.show()

# Publiser datastory

In [ ]:
from datastory import DataStory

In [ ]:
ds = DataStory(f"Automatiserings- og selvbetjeningsgrad for alderspensjon")

ds.markdown("**(Under utvikling)**")
ds.header(f"Automatisering av førstegangsbehandlingssaker for alderspensjon i pesys", level=2)
ds.markdown(f"Denne figuren gjelder kun førstegangsbehandling av alderspensjonssaker. Saker opprettet av batch er eksludert. For noen saker er deler av behandlingen automatisert, men disse regnes som manuelt behandlet.")
ds.plotly(figs["autograd"].to_json())

ds.header(f"Selvbetjening av førstegangsbehandlingssaker for alderspensjon i pesys", level=2)
ds.markdown(f"En sak regnes som selvbetjent dersom den er opprettet av en innbygger (ikke saksbehandler/batch). Denne figuren gjelder kun førstegangsbehandling av alderspensjonssaker. Saker opprettet av batch er eksludert.")
ds.plotly(figs["selvbetjening"].to_json())

ds.header(f"Blir saker opprettet av bruker automatisert? (Batch eksludert)", level=2)
ds.markdown(f"Automatisering av saker opprettet av bruker")
ds.plotly(figs["selv_auto"].to_json())
ds.markdown(f"Automatisering av saker opprettet av saksbehandler")
ds.plotly(figs["ikkeselv_auto"].to_json())

ds.header(f"Hvordan påvirker utlandstilsnitt automatiseringsgrad ved førstegangsbehandling?", level=2)
ds.markdown(f"Utlandstilsnitt på *manuelt* behandlede saker.")
ds.plotly(figs["man_utland"].to_json())
ds.markdown(f"Utlandstilsnitt på *automatisk* behandlede saker.")
ds.plotly(figs["auto_utland"].to_json())

ds.header(f"Hvor mange saker opprettet av batch behandles automatisk?")
ds.markdown(f"*Manuelt* behandlede saker")
ds.plotly(figs["man_batch"].to_json())
ds.markdown(f"*Automatisk* behandlede saker")
ds.plotly(figs["auto_batch"].to_json())


In [ ]:
ds.publish(url="https://nada.intern.nav.no/api") #prod

In [ ]:
#ds.update(token='77e7eb36-5a5b-4a9e-b6f9-ccee1c5db2e3')